In [8]:
from datetime import date
import json
from pathlib import Path

import geopandas as gpd
import pandas as pd

from hidromet.INMET import INMET
from hidromet import config
from hidromet import contornos
from hidromet import limpeza
from hidromet import modelos

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Obter inventário de postos do INMET

In [5]:
inmet = INMET()
estacoes = inmet.listar_estacoes()
df_estacoes = pd.DataFrame(estacoes)

df_estacoes.loc[:, ["VL_LATITUDE", "VL_LONGITUDE"]] = df_estacoes.loc[:, ["VL_LATITUDE", "VL_LONGITUDE"]].apply(pd.to_numeric)

# Recortar postos dentro do contorno das bacias

## Postos do INMET dentro dos contornos

In [6]:
gdf = gpd.read_file(config.dir_contorno)

coordenadas = contornos.zipar_coordenadas(latitude=df_estacoes.VL_LATITUDE, longitude=df_estacoes.VL_LONGITUDE)
postos_plu = contornos.criar_geodataframe(coordenadas=coordenadas, df=df_estacoes)
postos_plu.set_index("CD_ESTACAO", inplace=True)

geometria = gdf.iloc[0].geometry
bacia = gdf.iloc[0].bacia
postos = postos_plu.geometry

postos_inmet_dentro = contornos.recortar_postos(postos=postos, contorno=geometria)

## Obter séries de chuva por bacia

In [9]:
arquivo_bacia = config.dir_prec_inmet.joinpath(f"{bacia}.csv")
data_inicial = "2000-01-01" 
hoje = date.today().strftime("%Y-%m-%d")

bacia_df = limpeza.remover_codigos_duplicados(postos_inmet_dentro)
n_postos = len(bacia_df.index)

chuvas_bacia = list()
postos_sem_dado = list()
postos_com_falha = list()
postos_nao_representativos = list()
postos_ok = list()
for estacao in bacia_df.index:
    coordenadas = bacia_df.loc[estacao, 'geometry']
    estacao_json = inmet.obter_dados_estacao(cod_estacao=estacao, data_inicial=data_inicial, data_final=hoje, freq="D")
    df = pd.DataFrame(estacao_json)
    df.rename({"DT_MEDICAO": "data", "CHUVA": estacao}, axis=1, inplace=True)
    df.set_index("data", inplace=True)
    df.index = pd.to_datetime(df.index)

    df = df[[estacao]].apply(pd.to_numeric)

    df = limpeza.remover_duplicados(df)
    df.dropna(inplace=True)
    
    df[estacao] = limpeza.remover_outliers(df[estacao], p=0.99)
    df[estacao] = limpeza.substituir_negativos(df[estacao])
    falhas = limpeza.contabilizar_falhas(df[estacao])
    df = limpeza.remover_meses_nao_representativos(df)
    anos_disponiveis = limpeza.anos_disponiveis(df)
    n_dados = len(df)

    info_posto = modelos.Posto(
        latitude=coordenadas.y,
        longitude=coordenadas.x,
        codigo=estacao,
        n_dados=n_dados
    )


    if not df.empty and falhas < 0.15 and anos_disponiveis > 8:
        postos_ok.append(info_posto)
        chuvas_bacia.append(df)
    elif df.empty:
        postos_sem_dado.append(info_posto)
    elif anos_disponiveis < 8: 
        postos_nao_representativos.append(info_posto)
    elif falhas > 0.15:
        postos_com_falha.append(info_posto)


info_bacia = modelos.Bacia(
    serie_vazia=postos_sem_dado,
    nao_representativos=postos_nao_representativos,
    com_falhas=postos_com_falha,
    postos_ok=postos_ok,
    n_postos=n_postos,
    bacia=bacia
)


df_chuva = pd.concat(chuvas_bacia, axis=1)
df_chuva.to_csv(arquivo_bacia)

arquivo_json = Path(config.dir_prec_inmet, f'coords_{bacia}.json')
with open(arquivo_json, 'w') as f:
    json.dump(info_bacia.dict(), f)